# Haritha Thipparapu

## Week 10 - Extra Credit Online Retraining

### Implement Retraining Function (KNN or Dummy Classifier with Stratification

In [47]:
pip install fastapi

Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install fastapi uvicorn scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


In [49]:
import pandas as pd
import numpy as np
from typing import Optional
from fastapi import FastAPI
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report
import matplotlib
import matplotlib.pyplot as plt
import sklearn

In [58]:
app = FastAPI()

_x = [1]
_y = [1]
_result = ["True"]
_prediction = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x, _y]).reshape(1, -1), _result)

def model_evaluation(y_test, y_pred):
    testacc = accuracy_score(y_test, y_pred)
    testrecall = recall_score(y_test, y_pred, pos_label='True')
    testprecision = precision_score(y_test, y_pred, pos_label='True')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

@app.get("/predict/")
def predict(id: int = 0, x: float = 0, y: float = 0):
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x, y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    return {"prediction": prediction[0]}

@app.get("/result/")
def result(id: int, result: str):
    _result.insert(id, result)
    return {"message": f"Copied result for {id}"}

@app.get("/retrain/")
def retrain(last_n: int):
    global model
    print(f"RETRAINING over last {last_n}")
    n = min(last_n, len(_x))
    X = np.array(list(zip(_x[-n:], _y[-n:])))
    Y = np.array(_result[-n:])
    stratify = Y if len(set(Y)) > 1 else None
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, stratify=stratify)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)
    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model_type: str, last_n: int, strategy: Optional[str] = None, k: Optional[int] = 3):
    global model
    if model_type == "Dummy":
        model = DummyClassifier(strategy=strategy)
    elif model_type == "KNN":
        model = KNeighborsClassifier(n_neighbors=k)
    else:
        return {"error": "Unsupported model type"}
    message = retrain(last_n=last_n)
    return {"message": f"Model trained: {model}", "train": message}

In [60]:
# Preload synthetic data
for i in range(50):
    _x.append(i)
    _y.append(i % 2)
    _result.append(str(i % 3 == 0))

@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}


### the above didnt work for me so I tried the below one

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

def retrain_classifier(X, y, classifier_type='knn', n_neighbors=5, test_size=0.2, random_state=42):
    """
    Retrain a classifier (KNN or Dummy) using a stratified train-test split.

    Parameters:
        X (array-like): Feature matrix.
        y (array-like): Target labels (must be categorical for classification).
        classifier_type (str): 'knn' or 'dummy'.
        n_neighbors (int): Number of neighbors for KNN (ignored for dummy).
        test_size (float): Proportion of data for testing.
        random_state (int): Seed for reproducibility.

    Returns:
        model: The trained classifier.
        metrics (dict): Accuracy, precision, and recall scores.
    """
    # Stratified split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state)

    # Choose classifier
    if classifier_type == 'knn':
        model = KNeighborsClassifier(n_neighbors=n_neighbors)
    elif classifier_type == 'dummy':
        model = DummyClassifier(strategy='most_frequent')
    else:
        raise ValueError("classifier_type must be either 'knn' or 'dummy'")

    # Train and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='macro'),
        'recall': recall_score(y_test, y_pred, average='macro')
    }

    return model, metrics

In [65]:
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import KBinsDiscretizer

# Load diabetes data and convert to classification target
X, y = load_diabetes(return_X_y=True)
y_bin = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='quantile').fit_transform(y.reshape(-1, 1)).ravel()

# Retrain using KNN
model_knn, metrics_knn = retrain_classifier(X, y_bin, classifier_type='knn')
print("KNN Retraining Performance:", metrics_knn)

# Retrain using Dummy
model_dummy, metrics_dummy = retrain_classifier(X, y_bin, classifier_type='dummy')
print("Dummy Retraining Performance:", metrics_dummy)


KNN Retraining Performance: {'accuracy': 0.7528089887640449, 'precision': 0.7527777777777778, 'recall': 0.7527777777777778}
Dummy Retraining Performance: {'accuracy': 0.4943820224719101, 'precision': 0.24719101123595505, 'recall': 0.5}


C:\Users\harit\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
